In [53]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

In [54]:
print(sys.path)

['D:\\Repositories\\AI-project-methodology\\notebooks', 'D:\\Repositories\\AI-project-methodology\\env\\python310.zip', 'D:\\Repositories\\AI-project-methodology\\env\\DLLs', 'D:\\Repositories\\AI-project-methodology\\env\\lib', 'D:\\Repositories\\AI-project-methodology\\env', '', 'D:\\Repositories\\AI-project-methodology\\env\\lib\\site-packages', 'D:\\Repositories\\AI-project-methodology\\env\\lib\\site-packages\\win32', 'D:\\Repositories\\AI-project-methodology\\env\\lib\\site-packages\\win32\\lib', 'D:\\Repositories\\AI-project-methodology\\env\\lib\\site-packages\\Pythonwin', '..', '..', '..']


In [55]:
df_master= pd.read_csv(r'/Repositories/AI-project-methodology/data/train.csv',index_col='ID')

In [56]:
df = df_master.copy()
target=df_master['y']
df=df_master.drop('y',axis=1)
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
campaign     0
pdays        0
previous     0
poutcome     0
dtype: int64

# Creating Data PipeLines

In [57]:
continuous_columns = list(df.select_dtypes(include='number').columns)
categorical_columns=list(df.select_dtypes(include='object').columns)

## Continious

In [58]:
def impute_continuous_missing_values(dataframe: pd.DataFrame) -> pd.DataFrame:
    default_value = 0
    continuous_columns = dataframe.select_dtypes(include='number').columns
    [dataframe[column_name].fillna(default_value, inplace=True) for column_name in continuous_columns]
    return dataframe

def preprocess_continuous_data(dataframe: pd.DataFrame) -> pd.DataFrame:
     return impute_continuous_missing_values(dataframe)




In [59]:
preprocess_continuous_data(df)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
ID,,,,,,,,,,,,,,,
13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown
22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown
10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown
13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown
11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14023,47,technician,married,secondary,no,1167,yes,no,cellular,30,apr,1,87,5,failure
17259,31,unknown,married,secondary,no,111,no,no,cellular,21,nov,2,93,2,failure
15200,37,unemployed,single,tertiary,no,1316,yes,no,cellular,18,nov,1,172,2,failure


## Categorical

In [62]:
def get_categorical_column_names(dataframe: pd.DataFrame) -> [str]:
    categorical_columns = list(dataframe.select_dtypes(include='object').columns)
    return categorical_columns

def impute_categorical_missing_values(dataframe: pd.DataFrame) -> pd.DataFrame:
    default_value = 'missing value mate !'
    categorical_columns = get_categorical_column_names(dataframe)
    [dataframe[column_name].fillna(default_value, inplace=True) for column_name in categorical_columns]
    return dataframe

In [63]:
df_master[categorical_columns].isna().sum()


job          0
marital      0
education    0
default      0
housing      0
loan         0
contact      0
month        0
poutcome     0
dtype: int64

In [64]:
impute_categorical_missing_values(df)[categorical_columns].isna().sum()

job          0
marital      0
education    0
default      0
housing      0
loan         0
contact      0
month        0
poutcome     0
dtype: int64

In [65]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore', dtype=int, sparse=True)
encoder.fit(df[categorical_columns])
encoder

OneHotEncoder(dtype=<class 'int'>, handle_unknown='ignore')

In [66]:
encoded_categorical_df = encoder.transform(df[categorical_columns])
encoded_categorical_df 

<12870x44 sparse matrix of type '<class 'numpy.int32'>'
	with 115830 stored elements in Compressed Sparse Row format>

In [67]:
MODEL_DIR=Path(r'/Repositories/AI-project-methodology/models')

In [68]:
import joblib

def get_one_encoder(dataframe: pd.DataFrame, encoder_dir: Path) -> OneHotEncoder:
    one_hot_encoder_path = MODEL_DIR/'one_hot_encoder.joblib'
    if one_hot_encoder_path.exists():
        one_hot_encoder = joblib.load(one_hot_encoder_path)
    else:
        categorical_columns = get_categorical_column_names(dataframe)
        one_hot_encoder = OneHotEncoder(handle_unknown='ignore', dtype=int, sparse=True)
        one_hot_encoder.fit(dataframe[categorical_columns])
        joblib.dump(one_hot_encoder, one_hot_encoder_path) 
    return one_hot_encoder

def encode_categorical_data(dataframe: pd.DataFrame, one_hot_encoder: OneHotEncoder) -> pd.DataFrame:
    categorical_columns = get_categorical_column_names(dataframe)
    encoded_categorical_data_matrix = one_hot_encoder.transform(dataframe[categorical_columns])
    encoded_data_columns = one_hot_encoder.get_feature_names(categorical_columns)
    encoded_categorical_data_df = pd.DataFrame.sparse.from_spmatrix(data=encoded_categorical_data_matrix,
                                                                columns=encoded_data_columns, index=dataframe.index)
    encoded_df = dataframe.copy().drop(categorical_columns, axis=1).join(encoded_categorical_data_df)
    return encoded_df

In [69]:
encoder = get_one_encoder(df,MODEL_DIR)
encoded_df = encode_categorical_data(df, encoder)


D:\Repositories\AI-project-methodology\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [70]:
encoded_df


,age,balance,day,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
ID,,,,,,,,,,,,,,,,,,,,,
13829,29,18254,11,2,-1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
22677,26,512,5,3,-1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
10541,30,135,14,2,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13689,41,30,10,1,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
11304,27,321,2,1,-1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14023,47,1167,30,1,87,5,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
17259,31,111,21,2,93,2,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
15200,37,1316,18,1,172,2,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [71]:
def preprocess_categorical_data(dataframe: pd.DataFrame, models_dir: Path) -> pd.DataFrame:
    dataframe = impute_categorical_missing_values(dataframe)
    encoder = get_one_encoder(dataframe, models_dir)
    encoded_df = encode_categorical_data(dataframe, encoder)
    return encoded_df

In [72]:
def preprocess(dataframe: pd.DataFrame, models_dir) -> pd.DataFrame:
    preprocessed_df = dataframe.copy()
    preprocessed_df = preprocess_continuous_data(dataframe)
    preprocessed_df = preprocess_categorical_data(preprocessed_df, models_dir)
    return preprocessed_df

In [73]:
df_encoded = preprocess_categorical_data(df, MODEL_DIR)

D:\Repositories\AI-project-methodology\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [74]:
preprocessed_df = preprocess(df, MODEL_DIR)
preprocessed_df.head()

D:\Repositories\AI-project-methodology\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,age,balance,day,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
ID,,,,,,,,,,,,,,,,,,,,,
13829,29,18254,11,2,-1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
22677,26,512,5,3,-1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
10541,30,135,14,2,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13689,41,30,10,1,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
11304,27,321,2,1,-1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


# Modeling

In [75]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,scale, StandardScaler

In [76]:
x_tr,x_val,y_tr,y_val =train_test_split(df,target,test_size=0.3,random_state=42,stratify=target)

In [78]:
rf = RandomForestClassifier()
preprocessed_df = preprocess(x_tr, MODEL_DIR)
rf.fit(preprocessed_df, y_tr)

D:\Repositories\AI-project-methodology\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
D:\Repositories\AI-project-methodology\env\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


RandomForestClassifier()

In [79]:
preprocess_df_val=preprocess(x_val,MODEL_DIR)
pred = rf.predict_proba(preprocess_df_val)

D:\Repositories\AI-project-methodology\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
D:\Repositories\AI-project-methodology\env\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [83]:
from joblib import dump
dump(rf, MODEL_DIR/'predict.joblib')

['\\Repositories\\AI-project-methodology\\models\\predict.joblib']

In [81]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred[:,1])

0.7794587996186868